In [1]:
from pandas_datareader import data
import pandas_datareader as pdr
import pandas as pd
import numpy as np
#import talib as ta
import pandas_ta as ta
import datetime
import requests

In [5]:
ticker = 'ethbtc'
base = 'https://api.huobi.pro'
url = '/market/history/kline?period=4hour&size=2000&symbol=ethusdt'
final_url = base + url

reqs = requests.get(final_url)

df = pd.DataFrame(reqs.json()['data'])
df["Date"] = pd.to_datetime(df["id"], unit='s', origin='unix')
df["Date"] = df["Date"].dt.strftime("%m/%d/%Y %H:%M:%S")
df.drop('id',axis = 1, inplace = True)
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date',inplace = True)
df.rename(columns={'high':'High','low':'Low','open':'Open','close':'Close','vol':'Volume'},inplace=True)
df['Adj Close']= df['Close']
#df = df[['High','Low','Open','Close','Volume','Adj Close']]
df.sort_index(inplace = True)


In [7]:
df["sup"] = ta.supertrend(high = df["High"], low = df["Low"], close = df["Close"], period = 7 , mutiplier = 3)["SUPERT_7_3.0"]


,Open,Close,Low,High,amount,Volume,count,Adj Close,sup
Date,,,,,,,,,
2021-06-09 04:00:00,2455.23,2503.42,2403.44,2541.31,108054.388137,2.686623e+08,181246,2503.42,0.000000
2021-06-09 08:00:00,2503.42,2534.79,2468.06,2545.86,100407.534776,2.511655e+08,165358,2534.79,NaN
2021-06-09 12:00:00,2535.19,2566.72,2446.50,2579.90,179009.140155,4.530728e+08,287720,2566.72,NaN
2021-06-09 16:00:00,2566.73,2562.64,2549.00,2625.89,109565.463205,2.830704e+08,184528,2562.64,NaN
2021-06-09 20:00:00,2562.29,2610.37,2548.78,2626.00,51936.386313,1.344651e+08,104117,2610.37,NaN
...,...,...,...,...,...,...,...,...,...
2022-05-07 16:00:00,2692.58,2682.86,2680.24,2701.37,7999.313173,2.150916e+07,28740,2682.86,2821.099999
2022-05-07 20:00:00,2682.74,2635.78,2586.92,2682.95,28149.173331,7.404711e+07,71930,2635.78,2787.772142
2022-05-08 00:00:00,2635.78,2545.26,2517.65,2641.99,41338.835094,1.062564e+08,90295,2545.26,2764.111836


In [14]:
df["Buy Signal"] = 0
df["Sell Signal"] = 0 
n = 6 
for i in range (n,len(df)):
    if df["Close"][i-1] <= df["sup"][i-1] and df["Close"][i] > df["sup"][i]:
        df["Buy Signal"][i] = 1 
    if df["Close"][i-1] >= df["sup"][i-1] and df["Close"][i] > df["sup"][i]:
        df["Sell Signal"][i] = 1 

<ipython-input-14-2ea46b491c7f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Sell Signal"][i] = 1
<ipython-input-14-2ea46b491c7f>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Buy Signal"][i] = 1


In [15]:
df[(df['Buy Signal']>0) | df["Sell Signal"]>0]

,Open,Close,Low,High,amount,Volume,count,Adj Close,sup,Buy Signal,Sell Signal
Date,,,,,,,,,,,
2021-06-10 12:00:00,2566.20,2512.52,2496.24,2570.78,75396.979347,1.909283e+08,137681,2512.52,2294.189286,0,1
2021-06-10 16:00:00,2512.51,2458.64,2447.00,2523.38,64701.936276,1.601113e+08,136382,2458.64,2294.189286,0,1
2021-06-10 20:00:00,2458.64,2471.54,2426.31,2495.47,41456.022936,1.021580e+08,92231,2471.54,2294.189286,0,1
2021-06-11 00:00:00,2471.54,2465.47,2403.00,2498.02,70254.094594,1.716734e+08,143725,2465.47,2294.189286,0,1
2021-06-11 04:00:00,2465.38,2445.86,2416.16,2489.79,56691.235294,1.391278e+08,119007,2445.86,2294.189286,0,1
...,...,...,...,...,...,...,...,...,...,...,...
2022-05-04 16:00:00,2843.92,2955.35,2826.11,2966.56,34689.808399,9.999544e+07,120584,2955.35,2706.988346,1,0
2022-05-04 20:00:00,2955.74,2940.44,2921.11,2958.28,7300.424746,2.144764e+07,24792,2940.44,2761.467868,0,1
2022-05-05 00:00:00,2940.71,2941.79,2937.80,2954.00,5528.711987,1.628719e+07,18618,2941.79,2786.191030,0,1
